# Tools
## FunctionTools
FunctionTool allows to convert any Python function to a tool.

In [ ]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """
    Dummy method to get the weather for a given location.
    """
    
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny."

tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location."
)
tool.call("New York")

## QueryEngineTool
A QueryEngine can be transformed into a tool using the `QueryEngineTool` class.

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# this code is for HuggingFace
# from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
# from llama_index.embedding.huggingface import HuggingFaceEmbedding
# embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")
# llm = HuggingFAceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

embed_model = OllamaEmbedding(model_name="nomic-embed-text")

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = Ollama(model="qwen2.5-coder")
query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(query_engine, 
                                     name="Some useful tool name", 
                                     description="Some useful tool description")

tool.call("Respond using a persona that describes author and travel experiences?")

## ToolSpecs
`ToolSpecs` are collection of tools that work together harmoniously developed and maintained by the community. A `ToolSpec` combines related tools for specific purposes.

In [ ]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()

# look at the metadata of each tool
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

## UtilityTools
Tools to handle large amount of data from other tools. Oftentimes, directly querying an API can return an excessive amount of data, some of which may be irrelevant, overflow the context window of the LLM, or unnecessarily increase the number of tokens that you are using. The main two utility tools are:
- `OnDemandToolLoader`: turns an existing LLamaIndex data load into a tool that an agent can use
- `LoadAndSearchToolSpec`: takes in any existing Tool as input. As a tool spec, it implements to_tool_list, and when that function is called, two tools are returned: a loading tool and then a search tool. The load Tool execution would call the underlying Tool, and the index the output (by default with a vector index). The search Tool execution would take in a query string as input and call the underlying index.